<a href="https://colab.research.google.com/github/allfed/CropOpt/blob/standardize/notebooks/yield_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup (only run this section if using Colab)

In [ ]:
# We must clone the code repo into your Colab environment in order to use it.
# To do this we must first connect your GitHub account via this online command line.
# Run this block, and hit Enter on each prompt, not entering any text.
# This generates an SSH key which you'll add to your GitHub account.
!ssh-keygen -t rsa -b 4096

In [ ]:
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

In [ ]:
# This line prints the key. Copy the key to your clipboard.
!cat /root/.ssh/id_rsa.pub

In [ ]:
# Now, navigate to https://github.com/settings/keys and click "New SSH key"
# Copy paste the above into the "Key" field, and give it a title, e.g. "Colab key",
# then save the key. Once done, test using this line:
!ssh -T git@github.com

In [ ]:
## Now we can clone the repo using SSH
!rm -rf CropOpt/
!git clone -b standardize git@github.com:allfed/CropOpt.git

In [ ]:
# This adds the Python files in our CropOpt repo to our path so we can import
# them

import os
import sys

module_path = os.path.abspath(os.path.join('./CropOpt'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [ ]:
# Now we'll install other dependencies from our Poetry file. We do this by first
# installing the toml package so we can read the pyproject.toml file
!pip install toml

In [ ]:
# We then load the file and read our required packages from it
import toml
config = toml.load("CropOpt/pyproject.toml")
pip_packages = config["tool"]["poetry"]["dependencies"]

In [ ]:
# Finally, we install the packages

import subprocess
import sys

def install_pip_package(package, version):
    subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"])

for package, version in pip_packages.items():
    if package == "python":
        continue
    install_pip_package(package, version[1:])

# End of Colab-specific setup

# Yield calculation demo

The purpose of this file is to generate yields for spring wheat and spring barley, along with nutrition, and spits out a couple plots and a csv file at the end of the file. I can't emphasize enough that you should take a look at Params.ods  to understand the inputs to the model. Params.ods is a spreadsheet, you can open it with excel or open office. Also reach out to morgan@allfed.info if you have questions about this code.

First, let's import a few dependencies. 
 * Params: Custom script which imports parameters from Params.ods file
     * Params.ods is as spreadsheet file located here: https://github.com/allfed/CropOpt/Params.ods
     * based on Adin's collaboration document here: https://docs.google.com/spreadsheets/d/1Mh7bmZYHypN7bSmejCyu4hkbolQ3MRANc3LhflwRZoA/edit#gid=0
 * Plotter: utility for displaying geopandas maps and plots
 * pandas, geopandas: Data analysis and GIS packages
     * some basic things you can do with pandas: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
     * some basic things you can do with geopandas: https://geopandas.org/getting_started/introduction.html
     
 * OutdoorGrowth: Custom class which contains several useful functions for outdoor growth calculations
     * if you want to understand what the model is doing to calculate yields, you need to read through OutdoorGrowth code, located here:https://github.com/allfed/CropOpt/tree/master/Modules

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src import params
from src.plotter import Plotter
from src import outdoor_growth
from src.outdoor_growth import OutdoorGrowth
import pandas as pd
import geopandas as gpd

dir_path
/home/dmrivers/Code/CropOpt/src


Now, let's import a bunch of previously created geopandas datasets

In [3]:
params.importAll()


# NOTE - params.geopandasDataDir should add the appropriate root directory location
# to all the directories, whether in colab or on your local machine

#total solar flux at surface , W/m^2
fsns=pd.read_pickle(params.geopandasDataDir + 'FSNS.pkl')

#surface temperature, K
ts=pd.read_pickle(params.geopandasDataDir+'TS.pkl')

#Large-scale (stable) precipitation rate, including ice precipitate but not 
#snow, m/s
precl=pd.read_pickle(params.geopandasDataDir+'PRECL.pkl')

#absolute humidity at surface, kg/kg
q=pd.read_pickle(params.geopandasDataDir+'Q.pkl')

#crop area, Ha
growArea=pd.read_pickle(params.geopandasDataDir+'CropGrowHectares.pkl')

Taking a look at sun, temperature, and rain, we see latitude and longitude, each month after the winter is defined in a column, and the cell geometry is displayed (in this case, a 2 degree by 2 degree square). The first few cells are all at the south pole, so it's unsurprising that they show strange values.

The default months in the Params.ods are May year 5- April year 6 (first and second year of simulation). 

The average temperature coefficient hottest consecutive months required to grow each crop in each region are used to estimate temperature and rainfall growth parameters.

There's an issue with the import for april of year six, for temperature. 

In [4]:
fsns.head() #sun, W/m^2

,lats,lons,0005-05,0005-06,0005-07,0005-08,0005-09,0005-10,0005-11,0005-12,0006-01,0006-02,0006-03,0006-04,geometry
0,-90.0,-180.0,0.0,0.0,0.0,0.0,0.091110,8.981670,19.803129,45.104923,51.074444,21.573278,1.263111,0.0,"POLYGON ((-177.50000 -90.00000, -177.50000 -88..."
1,-90.0,-177.5,0.0,0.0,0.0,0.0,0.091316,9.011499,19.886478,45.310287,51.391033,21.682386,1.268457,0.0,"POLYGON ((-175.00000 -90.00000, -175.00000 -88..."
2,-90.0,-175.0,0.0,0.0,0.0,0.0,0.092592,9.066218,19.961117,45.563583,51.761959,21.826200,1.275466,0.0,"POLYGON ((-172.50000 -90.00000, -172.50000 -88..."
3,-90.0,-172.5,0.0,0.0,0.0,0.0,0.091346,8.990930,19.815718,45.142704,51.126923,21.593445,1.264396,0.0,"POLYGON ((-170.00000 -90.00000, -170.00000 -88..."
4,-90.0,-170.0,0.0,0.0,0.0,0.0,0.091314,8.990105,19.817955,45.147705,51.137344,21.594503,1.264395,0.0,"POLYGON ((-167.50000 -90.00000, -167.50000 -88..."


In [5]:
ts.head() #temp, kelvin

,lats,lons,0005-05,0005-06,0005-07,0005-08,0005-09,0005-10,0005-11,0005-12,0006-01,0006-02,0006-03,0006-04,geometry
0,-90.0,-180.0,220.342468,215.503311,220.463852,221.868210,223.157059,223.557083,230.742920,238.620422,238.959991,229.481277,224.424927,0.0,"POLYGON ((-177.50000 -90.00000, -177.50000 -88..."
1,-90.0,-177.5,220.340179,215.495789,220.467178,221.873779,223.160843,223.570450,230.769943,238.669510,239.032562,229.503387,224.421722,0.0,"POLYGON ((-175.00000 -90.00000, -175.00000 -88..."
2,-90.0,-175.0,220.370514,215.534012,220.475540,221.894608,223.168854,223.588333,230.771729,238.680923,239.090103,229.568085,224.460007,0.0,"POLYGON ((-172.50000 -90.00000, -172.50000 -88..."
3,-90.0,-172.5,220.354309,215.517487,220.456039,221.882523,223.154953,223.567429,230.734650,238.613739,238.971558,229.504593,224.440613,0.0,"POLYGON ((-170.00000 -90.00000, -170.00000 -88..."
4,-90.0,-170.0,220.349030,215.508636,220.457947,221.881195,223.156204,223.568954,230.742767,238.625778,238.979492,229.497925,224.433029,0.0,"POLYGON ((-167.50000 -90.00000, -167.50000 -88..."


In [6]:
precl.head() #rain, average m/s over the month

,lats,lons,0005-05,0005-06,0005-07,0005-08,0005-09,0005-10,0005-11,0005-12,0006-01,0006-02,0006-03,0006-04,geometry
0,-90.0,-180.0,2.036849e-09,1.260919e-09,2.828056e-09,2.751045e-09,3.160088e-09,2.173264e-09,7.796936e-10,6.155515e-10,5.939513e-10,8.130316e-10,2.176508e-09,4.003180e-09,"POLYGON ((-177.50000 -90.00000, -177.50000 -88..."
1,-90.0,-177.5,2.036849e-09,1.260919e-09,2.828056e-09,2.751045e-09,3.160088e-09,2.173264e-09,7.796936e-10,6.155515e-10,5.939513e-10,8.130316e-10,2.176508e-09,4.003180e-09,"POLYGON ((-175.00000 -90.00000, -175.00000 -88..."
2,-90.0,-175.0,2.036849e-09,1.260919e-09,2.828056e-09,2.751045e-09,3.160088e-09,2.173264e-09,7.796936e-10,6.155515e-10,5.939513e-10,8.130316e-10,2.176508e-09,4.003180e-09,"POLYGON ((-172.50000 -90.00000, -172.50000 -88..."
3,-90.0,-172.5,2.036849e-09,1.260919e-09,2.828056e-09,2.751045e-09,3.160088e-09,2.173264e-09,7.796936e-10,6.155515e-10,5.939513e-10,8.130316e-10,2.176508e-09,4.003180e-09,"POLYGON ((-170.00000 -90.00000, -170.00000 -88..."
4,-90.0,-170.0,2.036849e-09,1.260919e-09,2.828056e-09,2.751045e-09,3.160088e-09,2.173264e-09,7.796936e-10,6.155515e-10,5.939513e-10,8.130316e-10,2.176508e-09,4.003180e-09,"POLYGON ((-167.50000 -90.00000, -167.50000 -88..."


The grow area is shown below using hectares. Note it is only calculated for one month as it's assumed to be unchanging for now.

In [7]:
growArea.head()

,lats,lons,fraction,geometry,cellArea,growArea
0,-90.0,-180.0,0.0,"POLYGON ((-177.50000 -90.00000, -177.50000 -88...",97701.657432,0.0
1,-90.0,-177.5,0.0,"POLYGON ((-175.00000 -90.00000, -175.00000 -88...",97701.657432,0.0
2,-90.0,-175.0,0.0,"POLYGON ((-172.50000 -90.00000, -172.50000 -88...",97701.657432,0.0
3,-90.0,-172.5,0.0,"POLYGON ((-170.00000 -90.00000, -170.00000 -88...",97701.657432,0.0
4,-90.0,-170.0,0.0,"POLYGON ((-167.50000 -90.00000, -167.50000 -88...",97701.657432,0.0


In [8]:
q.head()

,lats,lons,0005-05,0005-06,0005-07,0005-08,0005-09,0005-10,0005-11,0005-12,0006-01,0006-02,0006-03,0006-04,geometry
0,-90.0,-180.0,0.00004,0.000022,0.000047,0.000046,0.000085,0.000071,0.000102,0.000226,0.000234,0.000096,0.000065,0.000092,"POLYGON ((-177.50000 -90.00000, -177.50000 -88..."
1,-90.0,-177.5,0.00004,0.000022,0.000047,0.000046,0.000085,0.000071,0.000102,0.000226,0.000234,0.000096,0.000065,0.000092,"POLYGON ((-175.00000 -90.00000, -175.00000 -88..."
2,-90.0,-175.0,0.00004,0.000022,0.000047,0.000046,0.000085,0.000071,0.000102,0.000226,0.000234,0.000096,0.000065,0.000092,"POLYGON ((-172.50000 -90.00000, -172.50000 -88..."
3,-90.0,-172.5,0.00004,0.000022,0.000047,0.000046,0.000085,0.000071,0.000102,0.000226,0.000234,0.000096,0.000065,0.000092,"POLYGON ((-170.00000 -90.00000, -170.00000 -88..."
4,-90.0,-170.0,0.00004,0.000022,0.000047,0.000046,0.000085,0.000071,0.000102,0.000226,0.000234,0.000096,0.000065,0.000092,"POLYGON ((-167.50000 -90.00000, -167.50000 -88..."


Let's also look at the values for some of the Params.ods variable values. It's important to be able to modify these if you want to be able to improve the cropOpt model. Make sure to restart the Jupyter kernel to reimport changes to params variables. The code below is looping through all the cells and all the months, so it might take a few minutes, depending on your machine.

Again, there appear to be issues with april of year 6 (second year of winter), and also possibly september year 5 (first year of winter).

In [9]:
allMonths=params.allMonths

#initialize object of outdoor growth class
outdoorGrowth=OutdoorGrowth()

In [10]:
plotYields = False # if plotYields is False, figures are saved but not shown

yields = outdoorGrowth.estimateYields(ts,precl,growArea,plotYields)

/home/dmrivers/.cache/pypoetry/virtualenvs/co--V6REObM-py3.7/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Month 0005-05 yields: 966170636379.4714
Month 0005-06 yields: 1550455454189.756
Month 0005-07 yields: 781913454539.0902
Month 0005-08 yields: 1833600941680.878
Month 0005-09 yields: 0.0
Month 0005-10 yields: 16513974610.77304
Month 0005-11 yields: 4481420057.670291
Month 0005-12 yields: 111893887446.69637
Month 0006-01 yields: 494484711811.82635
Month 0006-02 yields: 310378116122.90063
Month 0006-03 yields: 277840351592.36615
Month 0006-04 yields: 0


/home/dmrivers/.cache/pypoetry/virtualenvs/co--V6REObM-py3.7/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Month 0005-05 yields: 774682377472.7759
Month 0005-06 yields: 950790202534.0825
Month 0005-07 yields: 499970896273.77423
Month 0005-08 yields: 1084171851025.0289
Month 0005-09 yields: 2279062.4646386397
Month 0005-10 yields: 11586404120.152742
Month 0005-11 yields: 2266862265.5774455
Month 0005-12 yields: 65118904885.19839
Month 0006-01 yields: 277593620282.3057
Month 0006-02 yields: 188204177352.97446
Month 0006-03 yields: 177204262563.0798
Month 0006-04 yields: 0


Plots above are saved, see below for details

Below, we generate the remaining plots of the figures for all months of the simulation. This will take several minutes to run.

In [ ]:
#if plot booleans are False, figures are saved but not shown
plotSun=False
plotTemp=False
plotRain=False
plotHum=False 

print('Producing Sun Figures')
for month in allMonths:
	title="Solar Flux at Surface, month "+month
	legendlabel="Solar Flux (W/m^2)"
	fn="SolarFlux"+month
	Plotter.plotMap(fsns,month,title,legendlabel,fn,plotSun)

print('Producing Temp Figures')
for month in allMonths:
	title="Surface (radiative) Temperature, month "+month
	legendlabel="Temperature (K)"
	fn="SurfaceTemp"+month
	Plotter.plotMap(ts,month,title,legendlabel,fn,plotTemp)

print('Producing Rain Figures')
for month in allMonths:
	title="Precipitation "+month
	legendlabel="Precipitation (m/s)"
	fn="Precipitation"+month
	Plotter.plotMap(precl,month,title,legendlabel,fn,plotRain)

print('Producing Humidity Figures')
for month in allMonths:
	title="Humidity "+month
	legendlabel="Humidity (kg/kg)"
	fn="Humidity"+month
	Plotter.plotMap(q,month,title,legendlabel,fn,plotHum)


Plots above are saved in the reports/figures directory (run code below for specific location)

If you're running colab, go to the left side panel and click on the folder icon (tooltip says "Files"). You can download the figures from there. 

In [ ]:
# run this cell to see location where figures were saved from running the cells above
print("Saved figures location: " + params.figuresDir)

In [ ]:
# If you're interested in reproducing a CSV of many of the variables, you can call the function below:
outdoorGrowth.saveLandSunHumRainCSV(ts,q,fsns,precl,growArea)